In [1]:
import os
import time
import torch
import pandas as pd
import bitsandbytes as bnb

from tqdm import tqdm
from torch.nn import DataParallel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {DEVICE} device")

using cuda device


In [1]:
import torch
print(torch.__version__)

2.3.1+cu121


In [2]:
def find_best_model(model_name):
    log_path = f'/project/lt200301-edubot/Capstone-TamTanai/logs/{model_name}.csv'
    log = pd.read_csv(log_path)
    step = int(log.iloc[log['eval_loss'].argmin()]['step'])
    return f'/project/lt200301-edubot/Capstone-TamTanai/models/{model_name}/checkpoint-{step}'


# SeaLLMs-v3-7B-Chat-finetune-dataset-v1 Llama-3-Typhoon-1.5-8B-finetune-dataset-v1
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-dataset-v1' # Llama-3-Typhoon-1.5-8B-finetune-v1-rslora
FIND_BEST_MODEL = True
MODEL_NAME_OR_PATH = f"/project/lt200301-edubot/Capstone-TamTanai/models/{MODEL_NAME}"
if FIND_BEST_MODEL:
    MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
EVALUATE_RESULT_PATH = f"/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_{MODEL_NAME}.csv"

print(f'Inferencing model {MODEL_NAME}')

# Load both LLM model and tokenizer
def load_LLM_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME_OR_PATH,
        quantization_config=bnb_config,
        local_files_only=True,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        use_cache=False
    )
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME_OR_PATH,
        padding_side="left",
        add_eos_token=True,
        add_bos_token=True,
    )
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

model, tokenizer = load_LLM_and_tokenizer()
model.config.use_cache = False

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = DataParallel(model).module


def generate_answer_with_timer(text: str):
    try:
        start_time = time.time()
        encoded_input = tokenizer.encode(text, return_tensors='pt', add_special_tokens=False).to(DEVICE)
        with torch.cuda.amp.autocast():
            generate_kwargs = dict(
                {"input_ids": encoded_input},
                do_sample=True,
                max_new_tokens=512,
                top_p=0.9,
                temperature=0.7,
                repetition_penalty=1.1,
                use_cache=False,
                pad_token_id=tokenizer.eos_token_id,
            )
        encoded_output = model.generate(**generate_kwargs)
        response = tokenizer.decode(encoded_output[0][len(encoded_input[0]):], skip_special_tokens=True)
        response_time = time.time() - start_time
        return response, response_time
    except:
        print("Encounter error!")
        return None, 0.0


test_data = pd.read_csv("../asset/dataset/testdata.csv")
print("Finished reading data")

Inferencing model Llama-3-Typhoon-1.5-8B-finetune-dataset-v1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Let's use 4 GPUs!
Finished reading data


In [3]:
test_data

,question,answer,knowledges,prompt
0,ตามพระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545 ...,'สินค้า' หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพ...,พระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545\n\n...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...
1,หากลูกหนี้ไม่ได้กำหนดเวลาชำระหนี้ไว้ ลูกหนี้สา...,ลูกหนี้สามารถชำระหนี้ได้โดยพลัน,ประมวลกฎหมายแพ่งและพาณิชย์ - บรรพ 2 (หนี้) - ล...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...
2,เครื่องหมายการค้าที่จดทะเบียนไว้ตามกฎหมายเดิมจ...,เครื่องหมายการค้าที่จดทะเบียนไว้ตามพระราชบัญญั...,พระราชบัญญัติเครื่องหมายการค้า (ฉบับที่ 3) พ.ศ...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...
3,หากลูกหนี้ได้ชำระหนี้สินเต็มจำนวนแล้ว ศาลมีอำน...,ศาลมีอำนาจสั่งยกเลิกการล้มละลายหากหนี้สินของบุ...,พระราชบัญญัติล้มละลาย (ฉบับที่ 10) พ.ศ. 2561 -...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...
4,บัตรอิเล็กทรอนิกส์ตามนิยามในประมวลกฎหมายอาญาหม...,บัตรอิเล็กทรอนิกส์หมายถึงเอกสารหรือวัตถุอื่นใด...,ประมวลกฎหมายอาญา - ภาค 1 (บทบัญญัติทัว่ไป) - ล...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...
...,...,...,...,...
408,สถานศึกษา สถานฝึกและอบรม และสถานแนะนำทางจิตที่...,จะเป็นสถานศึกษา สถานฝึกและอบรม และสถานแนะนำทาง...,พระราชบัญญัติศาลเยาวชนและครอบครัวและวิธีพิจารณ...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...
409,หากผู้ทำแผนไม่สามารถมาประชุมได้เพราะเหตุผลพิเศ...,หากผู้ทำแผนไม่สามารถมาประชุมได้เพราะเหตุผลพิเศ...,พระราชบัญญัติล้มละลาย (ฉบับที่ 10) พ.ศ. 2561 -...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...
410,หากบุคคลใดกรอกข้อมูลเท็จลงในแบบสำรวจ จะต้องรับ...,บุคคลดังกล่าวจะต้องระวางโทษจำคุกไม่เกินหกเดือน...,พระราชบัญญัติประกันสังคม พ.ศ. 2533 (ฉบับปรับปร...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...
411,กรรมการค่าจ้างมีวาระการดำรงตำแหน่งกี่ปี?,กรรมการค่าจ้างมีวาระการดำรงตำแหน่งคราวละสองปี,พระราชบัญญัติคุ้มครองแรงงาน (ฉบับที่ 8) พ.ศ. 2...,<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบ...


In [4]:
test_data.iloc[365,-1]

'<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบคำถามเกี่ยวกับกฎหมาย จงตอบคำถามโดยใช้ความรู้ที่ให้ดังต่อไปนี้\nถ้าหากคุณไม่รู้คำตอบ ให้ตอบว่าไม่รู้ อย่าสร้างคำตอบขึ้นมาเอง\nความรู้ที่ให้:\nพระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565 - ลักษณะ 17 (เบ็ดเตล็ด)\n\nมาตรา 122  ผู้ขับขี่รถจักรยานยนต์และคนโดยสารรถจักรยานยนต์ต้องสวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตรายในขณะขับขี่และโดยสารรถจักรยานยนต์\nห้ามมิให้ผู้ขับขี่รถจักรยานยนต์ตามวรรคหนึ่งขับขี่รถจักรยานยนต์ในขณะที่คนโดยสารรถจักรยานยนต์มิได้สวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตราย\nลักษณะและวิธีการใช้หมวกเพื่อป้องกันอันตรายตามวรรคหนึ่ง ให้เป็นไปตามที่กำหนดในกฎกระทรวง\nความในวรรคหนึ่งมิให้ใช้บังคับแก่ภิกษุ สามเณร นักพรต นักบวช หรือผู้นับถือลัทธิศาสนาอื่นที่ใช้ผ้าหรือสิ่งอื่นโพกศีรษะตามประเพณีนิยมนั้น หรือบุคคลใดที่กำหนดในกฎกระทรวง</s><|im_start|>user\nหากผู้ขับขี่รถจักรยานยนต์ขับขี่โดยที่คนโดยสารไม่สวมหมวกนิรภัย จะถูกดำเนินการตามมาตราใด</s><|im_start|>assistant\n'

In [12]:
prompt = """'<s><|im_start|>system\nคุณคือนักกฎหมายที่จะตอบคำถามเกี่ยวกับกฎหมาย จงตอบคำถามโดยใช้ความรู้ที่ให้ดังต่อไปนี้\nถ้าหากคุณไม่รู้คำตอบ ให้ตอบว่าไม่รู้ อย่าสร้างคำตอบขึ้นมาเอง\nความรู้ที่ให้:\nพระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565 - ลักษณะ 17 (เบ็ดเตล็ด)\n\nมาตรา 122  ผู้ขับขี่รถจักรยานยนต์และคนโดยสารรถจักรยานยนต์ต้องสวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตรายในขณะขับขี่และโดยสารรถจักรยานยนต์\nห้ามมิให้ผู้ขับขี่รถจักรยานยนต์ตามวรรคหนึ่งขับขี่รถจักรยานยนต์ในขณะที่คนโดยสารรถจักรยานยนต์มิได้สวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตราย\nลักษณะและวิธีการใช้หมวกเพื่อป้องกันอันตรายตามวรรคหนึ่ง ให้เป็นไปตามที่กำหนดในกฎกระทรวง\nความในวรรคหนึ่งมิให้ใช้บังคับแก่ภิกษุ สามเณร นักพรต นักบวช หรือผู้นับถือลัทธิศาสนาอื่นที่ใช้ผ้าหรือสิ่งอื่นโพกศีรษะตามประเพณีนิยมนั้น หรือบุคคลใดที่กำหนดในกฎกระทรวง</s><|im_start|>user\nหากผู้ขับขี่รถจักรยานยนต์ขับขี่โดยที่คนโดยสารไม่สวมหมวกนิรภัย จะถูกดำเนินการตามมาตราใด</s><|im_start|>assistant\n'"""

In [13]:
print(prompt)

'<s><|im_start|>system
คุณคือนักกฎหมายที่จะตอบคำถามเกี่ยวกับกฎหมาย จงตอบคำถามโดยใช้ความรู้ที่ให้ดังต่อไปนี้
ถ้าหากคุณไม่รู้คำตอบ ให้ตอบว่าไม่รู้ อย่าสร้างคำตอบขึ้นมาเอง
ความรู้ที่ให้:
พระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565 - ลักษณะ 17 (เบ็ดเตล็ด)

มาตรา 122  ผู้ขับขี่รถจักรยานยนต์และคนโดยสารรถจักรยานยนต์ต้องสวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตรายในขณะขับขี่และโดยสารรถจักรยานยนต์
ห้ามมิให้ผู้ขับขี่รถจักรยานยนต์ตามวรรคหนึ่งขับขี่รถจักรยานยนต์ในขณะที่คนโดยสารรถจักรยานยนต์มิได้สวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตราย
ลักษณะและวิธีการใช้หมวกเพื่อป้องกันอันตรายตามวรรคหนึ่ง ให้เป็นไปตามที่กำหนดในกฎกระทรวง
ความในวรรคหนึ่งมิให้ใช้บังคับแก่ภิกษุ สามเณร นักพรต นักบวช หรือผู้นับถือลัทธิศาสนาอื่นที่ใช้ผ้าหรือสิ่งอื่นโพกศีรษะตามประเพณีนิยมนั้น หรือบุคคลใดที่กำหนดในกฎกระทรวง</s><|im_start|>user
หากผู้ขับขี่รถจักรยานยนต์ขับขี่โดยที่คนโดยสารไม่สวมหมวกนิรภัย จะถูกดำเนินการตามมาตราใด</s><|im_start|>assistant
'


In [14]:
output = generate_answer_with_timer(prompt)
output

("ห้ามมิให้ผู้ขับขี่รถจักรยานยนต์ตามวรรคหนึ่งขับขี่รถจักรยานยนต์ในขณะที่คนโดยสารรถจักรยานยนต์มิได้สวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตราย'</s>\n'ความในวรรคหนึ่งมิให้ใช้บังคับแก่ภิกษุ สามเณร นักพรต นักบวช หรือผู้นับถือลัทธิศาสนาอื่นที่ใช้ผ้าหรือสิ่งอื่นโพกศีรษะตามประเพณีนิยมนั้น หรือบุคคลใดที่กำหนดในกฎกระทรวง'</s>\n'ผู้ขับขี่รถจักรยานยนต์และคนโดยสารรถจักรยานยนต์ต้องสวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตรายในขณะขับขี่และโดยสารรถจักรยานยนต์'</s>\n'ห้ามมิให้ผู้ขับขี่รถจักรยานยนต์ตามวรรคหนึ่งขับขี่รถจักรยานยนต์ในขณะที่คนโดยสารรถจักรยานยนต์มิได้สวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตราย'</s>\n'ความในวรรคหนึ่งมิให้ใช้บังคับแก่ภิกษุ สามเณร นักพรต นักบวช หรือผู้นับถือลัทธิศาสนาอื่นที่ใช้ผ้าหรือสิ่งอื่นโพกศีรษะตามประเพณีนิยมนั้น หรือบุคคลใดที่กำหนดในกฎกระทรวง'</s>\n'ผู้ขับขี่รถจักรยานยนต์และคนโดยสารรถจักรยานยนต์ต้องสวมหมวกที่จัดทำขึ้นโดยเฉพาะเพื่อป้องกันอันตรายในขณะขับขี่และโดยสารรถจักรยานยนต์'</s>\n'ห้ามมิให้ผู้ขับขี่รถจักรยานยนต์ตามวรรคหนึ่งขับขี่รถจักรยานยนต์ในขณะที่คนโดยสารรถ